<a href="https://colab.research.google.com/github/alayuala/114-1PL.repo/blob/main/WEEK10_%E8%AA%B2%E8%A1%A8%E6%9F%A5%E8%A9%A2%E8%88%87%E6%8F%90%E9%86%92%E7%B4%A0%E6%9D%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **課表查詢與提醒素材（作業六）**
* 目標：從 Sheet 讀課表 → 指定星期輸出當日課程、地點 → 回寫「本週重點」分頁。
* AI 點子：請模型把本週課程壓成一句「行前提醒」（攜帶品/先讀章節）。


In [9]:
# --- ================================== ---
# 🚀 HW6 - 課表 AI 提醒器 (V6.1 - 排序 + Gradio 綁定修正)
# 最終合併版 (All-in-One)
# --- ================================== ---

# --- 1. 安裝所有必要的套件 ---
print("⏳ 正在安裝必要的套件 (gradio, gspread, google-generativeai...)...")
!pip install -q gradio gspread gspread_dataframe pandas google-auth google-auth-oauthlib google-auth-httplib2 google-generativeai pytz

print("✅ 所有套件安裝完成！")

# --- 2. 載入所有函式庫 ---
import os
import pandas as pd
import gspread
import gradio as gr
import google.generativeai as genai
import pytz
import json
from google.colab import auth, userdata
from google.auth import default
from gspread_dataframe import set_with_dataframe
from datetime import datetime, timedelta

print("✅ 函式庫載入完成。")

# --- 3. Google 認證 與 API 金鑰設定 ---

print("⏳ 正在進行 Google 認證...")
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("✅ Colab 帳號認證成功 (Google Sheet)。")
except Exception as e:
    print(f"❌ Colab 認證失敗：{e}")

try:
    GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel('gemini-2.5-flash')
    print("✅ Gemini AI 模型 (gemini-2.5-flash) 初始化成功。")
except Exception as e:
    print(f"❌ 找不到或無法設定 Gemini API 金鑰！(錯誤: {e})")
    model = None

# --- 4. 【V6】建立您的「真實 + 豐富資訊」課表 ---

SHEET_URL = "https://docs.google.com/spreadsheets/d/1JreQ6dAZCucHYB6XE5BL4dKEUcoDO-kcYQK33EN3Z38/edit?usp=sharing"
WORKSHEET_NAME = "HW6" # 原始課表
SUMMARY_SHEET_TEXT = "HW6_重點摘要"
SUMMARY_SHEET_ITEMS = "[攜帶物品]"
SUMMARY_SHEET_FOCUS = "[學習重點]"
PLAN_SHEET_NAME = "[學習計畫]"
TIMEZONE = "Asia/Taipei"
TW_TZ = pytz.timezone(TIMEZONE)

# (V4.1 的真實課表範本)
WEEKLY_SCHEDULE_TEMPLATE = {
    0: [{'課程名稱': '設計概論', '時間(起)': '09:10', '時間(迄)': '10:00', '地點': '美101'},
        {'課程名稱': '設計概論', '時間(起)': '10:20', '時間(迄)': '11:10', '地點': '美101'},
        {'課程名稱': '設計概論', '時間(起)': '11:20', '時間(迄)': '12:10', '地點': '美101'},
        {'課程名稱': '設計素描 (一)', '時間(起)': '13:20', '時間(迄)': '14:10', '地點': '設計系2A教室'},
        {'課程名稱': '設計素描 (一)', '時間(起)': '14:20', '時間(迄)': '15:10', '地點': '設計系2A教室'},
        {'課程名稱': '設計素描 (一)', '時間(起)': '15:20', '時間(迄)': '16:10', '地點': '設計系2A教室'},],
    1: [{'課程名稱': '室內設計圖學', '時間(起)': '08:10', '時間(迄)': '09:00', '地點': '設計系2A教室'},
        {'課程名稱': '室內設計圖學', '時間(起)': '09:10', '時間(迄)': '10:00', '地點': '設計系2A教室'},
        {'課程名稱': '室內設計 (一)', '時間(起)': '10:20', '時間(迄)': '11:10', '地點': '設計系2B教室'},
        {'課程名稱': '室內設計 (一)', '時間(起)': '11:20', '時間(迄)': '12:10', '地點': '設計系2B教室'},
        {'課程名稱': '色彩學', '時間(起)': '13:20', '時間(迄)': '14:10', '地點': '科技系TB213教室'},
        {'課程名稱': '色彩學', '時間(起)': '14:20', '時間(迄)': '15:10', '地點': '科技系TB213教室'},
        {'課程名稱': '色彩學', '時間(起)': '15:20', '時間(迄)': '16:10', '地點': '科技系TB213教室'},],
    2: [{'課程名稱': '基礎模型製作 (一)', '時間(起)': '13:20', '時間(迄)': '14:10', '地點': '設計系2A教室'},
        {'課程名稱': '基礎模型製作 (一)', '時間(起)': '14:20', '時間(迄)': '15:10', '地點': '設計系2A教室'},],
    3: [{'課程名稱': '程式語言', '時間(起)': '09:10', '時間(迄)': '10:00', '地點': '科技系TB311教室'},
        {'課程名稱': '程式語言', '時間(起)': '10:20', '時間(迄)': '11:10', '地點': '科技系TB311教室'},
        {'課程名稱': '程式語言', '時間(起)': '11:20', '時間(迄)': '12:10', '地點': '科技系TB311教室'},
        {'課程名稱': '基礎設計 (一)', '時間(起)': '13:20', '時間(迄)': '14:10', '地點': '設計系2A教室'},
        {'課程名稱': '基礎設計 (一)', '時間(起)': '14:20', '時間(迄)': '15:10', '地點': '設計系2A教室'},
        {'課程名稱': '基礎設計 (一)', '時間(起)': '15:20', '時間(迄)': '16:10', '地點': '設計系2A教室'},],
    4: [{'課程名稱': '產品表現技法 (一)', '時間(起)': '09:10', '時間(迄)': '10:00', '地點': '設計系2B教室'},
        {'課程名稱': '產品表現技法 (一)', '時間(起)': '10:20', '時間(迄)': '11:10', '地點': '設計系2B教室'},
        {'課程名稱': '產品表現技法 (一)', '時間(起)': '11:20', '時間(迄)': '12:10', '地點': '設計系2B教室'},
        {'課程名稱': '電腦輔助製圖', '時間(起)': '14:20', '時間(迄)': '15:10', '地點': '科技系TA309教室'},
        {'課程名稱': '電腦輔助製圖', '時間(起)': '15:20', '時間(迄)': '16:10', '地點': '科技系TA309教室'},
        {'課程名稱': '電腦輔助製圖', '時間(起)': '16:30', '時間(迄)': '17:20', '地點': '科技系TA309教室'},],
    5: [], 6: []
}
WEEKDAY_MAP = {0: '一', 1: '二', 2: '三', 3: '四', 4: '五', 5: '六', 6: '日'}

def generate_full_schedule_v2(start_date_str, end_date_str, schedule_template):
    """ (V4 版日曆產生器) 自動產生課表並注入特殊提醒 """
    print(f"⏳ (V4) 正在產生從 {start_date_str} 到 {end_date_str} 的「豐富版」課表...")
    all_classes = []
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d').date()
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d').date()

    NEXT_WED = datetime(2025, 11, 12).date()
    NEXT_THU = datetime(2025, 11, 13).date()

    current_date = start_date
    while current_date <= end_date:
        weekday_int = current_date.weekday()
        if weekday_int in schedule_template:
            day_schedule = schedule_template[weekday_int]

            items_today_all_day = []
            if weekday_int in [0, 1, 2, 3]: items_today_all_day.append("圖袋")
            if weekday_int == 3: items_today_all_day.append("電腦")
            if weekday_int == 4: items_today_all_day.append("電腦")

            for course in day_schedule:
                new_course = course.copy()
                items_today_course = items_today_all_day.copy()
                notes_today_course = []

                if weekday_int == 4 and "產品表現技法" in new_course["課程名稱"]:
                    items_today_course.append("素描鉛筆")
                if current_date == NEXT_WED and "基礎模型製作" in new_course["課程名稱"]:
                    items_today_course.append("模型")
                    notes_today_course.append("發表")
                if current_date == NEXT_THU and "基礎設計" in new_course["課程名稱"]:
                    items_today_course.append("模型")
                    notes_today_course.append("發表")

                final_items = ", ".join(sorted(list(set(items_today_course))))
                final_notes = ", ".join(sorted(list(set(notes_today_course))))

                new_course_entry = {
                    "日期": current_date.strftime('%Y-%m-%d'), "星期": WEEKDAY_MAP[weekday_int],
                    "課程名稱": new_course['課程名稱'], "時間(起)": new_course['時間(起)'],
                    "時間(迄)": new_course['時間(迄)'], "地點": new_course['地點'],
                    "攜帶品": final_items, "先讀章節": "", "備註": final_notes
                }
                all_classes.append(new_course_entry)

        current_date += timedelta(days=1)

    df_all_classes = pd.DataFrame(all_classes)
    df_final = df_all_classes.drop_duplicates(subset=["日期", "課程名稱", "時間(起)"], keep='first')
    print(f"✅ (V4) 成功產生 {len(df_final)} 筆「豐富版」課程資料。")
    return df_final

# --- 執行日曆產生器並寫入 GSheet ---
df_real_schedule = generate_full_schedule_v2(
    start_date_str="2025-11-03",
    end_date_str="2025-12-23",
    schedule_template=WEEKLY_SCHEDULE_TEMPLATE
)

try:
    gsheets = gc.open_by_url(SHEET_URL)
    worksheet = gsheets.worksheet(WORKSHEET_NAME)
    worksheet.clear()
    set_with_dataframe(worksheet, df_real_schedule, resize=True)
    print(f"✅ 成功將 {len(df_real_schedule)} 筆「V4 真實課表」資料覆蓋寫入 '{WORKSHEET_NAME}'！")
except gspread.exceptions.WorksheetNotFound:
    print(f"❌ 找不到 '{WORKSHEET_NAME}'，正在嘗試建立...")
    try:
        worksheet = gsheets.add_worksheet(title=WORKSHEET_NAME, rows=len(df_real_schedule)+1, cols=len(df_real_schedule.columns))
        set_with_dataframe(worksheet, df_real_schedule, resize=True)
        print(f"✅ 成功建立工作表並寫入 {len(df_real_schedule)} 筆資料。")
    except Exception as e:
        print(f"❌ 建立 GSheet 時發生錯誤：{e}")
except Exception as e:
    print(f"❌ 寫入 Google Sheet 時發生錯誤：{e}")


# --- 5. 核心功能函數 (Backend) ---

def get_current_week_range(tz_str=TIMEZONE):
    tz = pytz.timezone(tz_str)
    today = datetime(2025, 11, 6, tzinfo=tz).date() # 強制 11/06 (週四)
    print(f"ℹ️ (偵錯) 已強制設定今天日期為: {today}")
    monday = today - timedelta(days=today.weekday())
    sunday = monday + timedelta(days=6)
    return monday, sunday

def load_and_filter_schedule(gsheets, worksheet_name, week_start, week_end):
    """從 GSheet 載入並篩選出本週的課表"""
    try:
        ws = gsheets.worksheet(worksheet_name)
        df = pd.DataFrame(ws.get_all_records())

        # --- 【V6.1 修正 BUG 1：排序錯誤】 ---
        # 1. 轉換日期
        df['日期'] = pd.to_datetime(df['日期']).dt.date
        # 2. 強制將「時間(起)」欄位轉換為真正的 "Time" 物件
        #    errors='coerce' 會讓 GSheet 的空值或 'N/A' 轉為 NaT (Not a Time)
        df['時間(起)_dt'] = pd.to_datetime(df['時間(起)'], format='%H:%M', errors='coerce').dt.time

        # 3. 篩選本週
        mask = (df['日期'] >= week_start) & (df['日期'] <= week_end)

        # 4. 使用我們新的 '時間(起)_dt' 欄位來排序
        df_this_week = df[mask].sort_values(
            by=['日期', '時間(起)_dt'] # 依賴 Time 物件排序
        ).reset_index(drop=True)

        return df_this_week

    except Exception as e:
        print(f"❌ 讀取 '{worksheet_name}' 失敗: {e}")
        return pd.DataFrame()

def generate_weekly_summary_v4_JSON(df_week, model_instance):
    """ (V5.1 版) AI 提示 - 結構化 JSON 輸出 (for 2.5-flash) """
    if df_week.empty:
        empty_json = {"full_summary_text": "🎉 本週無課，好好享受！", "carrying_items_list": "無", "study_focus_list": "無", "classroom_reminders": "無", "study_suggestions": "無"}
        return json.dumps(empty_json)

    if not model_instance:
        error_json = {"full_summary_text": "AI 模型未啟動。", "carrying_items_list": "AI 錯誤", "study_focus_list": "AI 錯誤", "classroom_reminders": "AI 錯誤", "study_suggestions": "AI 錯誤"}
        return json.dumps(error_json)

    schedule_str_list = []
    for _, row in df_week.iterrows():
        # (V6.1 修正) 確保 row['時間(起)'] 是字串
        time_str = row['時間(起)']
        if not isinstance(time_str, str):
            try:
                time_str = time_str.strftime('%H:%M')
            except:
                time_str = str(row['時間(起)'])

        items = f"(攜帶: {row['攜帶品']})" if row['攜帶品'] else ""
        notes = f"(備註: {row['備註']})" if row['備註'] else ""
        schedule_str_list.append(f"  - {row['日期']} (星期{row['星期']}) {time_str}：{row['課程名稱']} @ {row['地點']} {items} {notes}")
    schedule_str = "\n".join(sorted(list(set(schedule_str_list))))

    system_prompt = (
        "你是一位活潑、幽默且細心的系辦助理 🎓。\n"
        "你的任務是根據使用者 (設計系學生) 提供的「本週課表」，總結出一份「本週重點提醒」。\n"
        "風格：請使用**繁體中文**，語氣要**輕鬆、鼓勵**，並**多加使用 Emoji**！\n"
        "***【重要】***: 你的回答**必須**是一個格式**嚴格**的 JSON 物件，絕對不能有任何 JSON 以外的文字 (例如 'Here is the JSON:' 這種開頭)。\n"
        "JSON 格式如下:\n"
        "{\n"
        "  \"full_summary_text\": \"(字串。這是一個完整的、格式優美的提醒，包含所有區塊，用 \\n 換行。例如：🚀 肝力滿滿的一週又開始啦！\\n\\n🎒 必帶物品\\n- 圖袋\\n- 電腦\\n\\n...)\",\n"
        "  \"carrying_items_list\": \"(字串。只包含不重複的「攜帶品」清單，用 , 分隔。例如：圖袋, 電腦, 素描鉛筆)\",\n"
        "  \"study_focus_list\": \"(字串。只包含不重複的「備註」清單，例如：發表)\",\n"
        "  \"classroom_reminders\": \"(字串。一句關於「地點」的提醒。例如：這週要跑『設計系』和『科技系』喔！)\",\n"
        "  \"study_suggestions\": \"(字串。一句基於課程名稱的「學習建議」。例如：這週有程式和素描，記得左右腦均衡一下！)\"\n"
        "}\n"
    )
    user_query = f"這是我本週的課表，請幫我生成「本週重點提醒」的 JSON 物件：\n\n{schedule_str}"

    try:
        response = model_instance.generate_content(
            [system_prompt, user_query],
            generation_config=genai.types.GenerationConfig(temperature=0.7)
        )
        response_text = response.text.strip().replace("```json\n", "").replace("\n```", "")
        return response_text

    except Exception as e:
        error_json = {"full_summary_text": f"AI 摘要產生失敗：{e}", "carrying_items_list": "AI 錯誤", "study_focus_list": "AI 錯誤", "classroom_reminders": "AI 錯誤", "study_suggestions": "AI 錯誤"}
        return json.dumps(error_json)

# --- 【V6】GSheet 儲存輔助函數 ---
def _append_to_sheet(gsheets, sheet_name, data_row):
    """輔助函數：開啟 GSheet 並追加一行"""
    try:
        try:
            summary_ws = gsheets.worksheet(sheet_name)
        except gspread.exceptions.WorksheetNotFound:
            summary_ws = gsheets.add_worksheet(title=sheet_name, rows=100, cols=5)
            if sheet_name == SUMMARY_SHEET_ITEMS:
                summary_ws.append_row(["週次範圍", "攜帶物品", "儲存時間"])
            elif sheet_name == SUMMARY_SHEET_FOCUS:
                summary_ws.append_row(["週次範圍", "學習重點", "教室提醒", "學習建議", "儲存時間"])
            elif sheet_name == PLAN_SHEET_NAME:
                summary_ws.append_row(["儲存時間", "我的目標", "AI 計畫"])
            else:
                summary_ws.append_row(["週次範圍", "AI 提醒 (全文)", "儲存時間"])

        summary_ws.append_row(data_row)
        return True
    except Exception as e:
        print(f"❌ 寫入 GSheet '{sheet_name}' 失敗: {e}")
        return False

# --- 【V6.1】Gradio 核心互動函數 (修正版) ---

def filter_schedule_by_day(selected_weekday, df_state):
    """(主頁) 根據選擇的星期篩選課表"""
    if df_state.empty: return pd.DataFrame()
    filtered = df_state[df_state['星期'] == selected_weekday]
    # (V6.1) 確保顯示時，欄位是我們想要的 (移除 _dt)
    return filtered[['課程名稱', '時間(起)', '時間(迄)', '地點', '攜帶品', '備註']]

def update_ai_summary_dynamically(df_state, selected_course, new_item, new_note):
    """(Tab 2) 動態更新 AI 摘要 (不儲存)"""
    if df_state.empty:
        new_json_str = generate_weekly_summary_v4_JSON(df_state, model)
    else:
        df_updated = df_state.copy()
        if selected_course:
            mask = df_updated['課程名稱'] == selected_course
            if new_item: df_updated.loc[mask, '攜帶品'] = df_updated.loc[mask, '攜帶品'].apply(lambda x: f"{x}, {new_item}" if x else new_item)
            if new_note: df_updated.loc[mask, '備註'] = df_updated.loc[mask, '備註'].apply(lambda x: f"{x}, {new_note}" if x else new_note)
        new_json_str = generate_weekly_summary_v4_JSON(df_updated, model)

    try:
        new_json_obj = json.loads(new_json_str)
        new_text = new_json_obj.get('full_summary_text', 'AI JSON 格式錯誤 (full_summary_text)')
    except json.JSONDecodeError:
        new_text = f"AI 回傳了非 JSON 格式：\n{new_json_str}"
        new_json_str = "{}"
    return new_text, new_json_str

def save_all_summaries(week_range_str, ai_summary_text_edited, ai_summary_json_str):
    """(主頁) 儲存「所有」AI 細項到 GSheet"""
    save_time = datetime.now(TW_TZ).strftime("%Y-%m-%d %H:%M")
    success_count = 0

    row_text = [week_range_str, ai_summary_text_edited, save_time]
    if _append_to_sheet(gsheets_global, SUMMARY_SHEET_TEXT, row_text): success_count += 1

    try:
        ai_json_obj = json.loads(ai_summary_json_str)

        row_items = [week_range_str, ai_json_obj.get('carrying_items_list', 'JSON 格式錯誤'), save_time]
        if _append_to_sheet(gsheets_global, SUMMARY_SHEET_ITEMS, row_items): success_count += 1

        row_focus = [
            week_range_str, ai_json_obj.get('study_focus_list', 'JSON 格式錯誤'),
            ai_json_obj.get('classroom_reminders', 'JSON 格式錯誤'),
            ai_json_obj.get('study_suggestions', 'JSON 格式錯誤'), save_time
        ]
        if _append_to_sheet(gsheets_global, SUMMARY_SHEET_FOCUS, row_focus): success_count += 1

        if success_count == 3:
            return "✅ 成功儲存「全文」、「攜帶物品」、「學習重點」到 3 個 GSheet 分頁！"
        else:
            return "❌ 儲存 GSheet 時發生部分錯誤，請檢查 Colab 輸出。"

    except json.JSONDecodeError as e:
        return f"❌ 儲存細項失敗：AI 回傳的 JSON 格式錯誤。 (錯誤: {e})"

def save_direct_entry(week_range_str, entry_type, entry_text):
    """(Tab 3) 快速新增 (Quick Add)"""
    if not entry_text:
        return "⚠️ 請輸入要新增的內容。"
    save_time = datetime.now(TW_TZ).strftime("%Y-%m-%d %H:%M")
    if entry_type == "item":
        row_data = [week_range_str, entry_text, save_time]
        if _append_to_sheet(gsheets_global, SUMMARY_SHEET_ITEMS, row_data):
            return f"✅ 已將「{entry_text}」儲存到 `[攜帶物品]` 分頁！"
    elif entry_type == "note":
        row_data = [week_range_str, entry_text, "(手動新增)", "(手動新增)", save_time]
        if _append_to_sheet(gsheets_global, SUMMARY_SHEET_FOCUS, row_data):
            return f"✅ 已將「{entry_text}」儲存到 `[學習重點]` 分頁！"
    return "❌ 儲存失敗，請檢查 Colab 輸出。"

def _get_ai_learning_helper(mode, course_name, user_goal):
    """ (V6.1) AI 學習計畫室的「原始」後端 (只回傳 1 個值) """
    if not model: return "AI 模型未啟動。"
    if mode == "YouTube":
        if not course_name: return "請先選擇一門課程。"
        system_prompt = "你是一位專業的線上課程助教。請根據使用者提供的「課程名稱」，推薦 2-3 個相關的 YouTube「中文」教學影片關鍵字或知名頻道。格式：請用條列式清單回覆。"
        user_query = f"課程名稱：{course_name}"
    elif mode == "Plan":
        if not user_goal: return "請先輸入您的學習目標。"
        system_prompt = "你是一位專業的學習教練。請根據使用者提供的「本週學習目標」，為他生成一個簡單、可執行的 3 步驟行動計畫。風格：繁體中文、條列式、語氣鼓勵。"
        user_query = f"我的目標：{user_goal}"
    else:
        return "錯誤的模式。"
    try:
        response = model.generate_content([system_prompt, user_query], generation_config=genai.types.GenerationConfig(temperature=0.7))
        return response.text.strip()
    except Exception as e:
        return f"AI 處理失敗：{e}"

# --- 【V6.1 修正 BUG 2：Gradio 綁定錯誤】 ---
# 建立新的「包裝函數」
def run_youtube_helper(course_name):
    """ (Tab 4) 包裝
    - 呼叫 AI
    - 只回傳 1 個值給 1 個輸出
    """
    return _get_ai_learning_helper("YouTube", course_name, None)

def run_plan_helper(user_goal):
    """ (Tab 4) 包裝
    - 呼叫 AI
    - 回傳 2 個值給 2 個輸出 (畫面 + 隱藏 State)
    """
    plan_text = _get_ai_learning_helper("Plan", None, user_goal)
    return plan_text, plan_text # 回傳兩次!

def save_learning_plan(goal, plan_text):
    """(Tab 4) 儲存學習計畫"""
    if not goal or not plan_text:
        return "⚠️ 沒有目標或計畫可以儲存。"
    save_time = datetime.now(TW_TZ).strftime("%Y-%m-%d %H:%M")
    row_data = [save_time, goal, plan_text]
    if _append_to_sheet(gsheets_global, PLAN_SHEET_NAME, row_data):
        return f"✅ 學習計畫已成功儲存至 `[學習計畫]` 分頁！"
    return "❌ 計畫儲存失敗，請檢查 Colab 輸出。"

# --- 6. Gradio 介面配置 (Frontend) ---

print("⏳ 正在準備 Gradio 應用程式...")

# 應用程式啟動時的全局變數
try:
    gsheets_global = gc.open_by_url(SHEET_URL)
    (WEEK_START, WEEK_END) = get_current_week_range()
    DF_THIS_WEEK_ORIGINAL = load_and_filter_schedule(gsheets_global, WORKSHEET_NAME, WEEK_START, WEEK_END)

    INITIAL_AI_SUMMARY_JSON_STR = generate_weekly_summary_v4_JSON(DF_THIS_WEEK_ORIGINAL, model)
    try:
        INITIAL_AI_SUMMARY_OBJ = json.loads(INITIAL_AI_SUMMARY_JSON_STR)
        INITIAL_AI_SUMMARY_TEXT = INITIAL_AI_SUMMARY_OBJ.get('full_summary_text', 'AI 摘要載入失敗 (JSON 格式錯誤)')
    except json.JSONDecodeError:
        INITIAL_AI_SUMMARY_TEXT = f"AI 摘要載入失敗 (AI 回傳非 JSON):\n{INITIAL_AI_SUMMARY_JSON_STR}"
        INITIAL_AI_SUMMARY_JSON_STR = "{}"

    WEEK_RANGE_STR = f"{WEEK_START.strftime('%Y-%m-%d')} ~ {WEEK_END.strftime('%Y-%m-%d')}"
    TODAY_WEEKDAY_STR = ["一", "二", "三", "四", "五", "六", "日"][3] # 強制週四
    WEEK_COURSES_LIST = DF_THIS_WEEK_ORIGINAL['課程名稱'].unique().tolist()

except Exception as e:
    print(f"❌ Gradio 啟動前置作業失敗: {e}")
    DF_THIS_WEEK_ORIGINAL = pd.DataFrame()
    INITIAL_AI_SUMMARY_TEXT, INITIAL_AI_SUMMARY_JSON_STR, WEEK_RANGE_STR = "錯誤", "{}", "N/A"
    TODAY_WEEKDAY_STR, WEEK_COURSES_LIST = "一", []

print("⏳ 正在啟動 Gradio 應用程式...")

with gr.Blocks(theme=gr.themes.Soft(), title="課表 AI 提醒器 V6.1") as demo:

    # 【V6】建立隱藏 State
    df_week_state = gr.State(DF_THIS_WEEK_ORIGINAL)
    ai_summary_json_state = gr.State(INITIAL_AI_SUMMARY_JSON_STR)
    week_range_state = gr.State(WEEK_RANGE_STR)
    plan_text_state = gr.State("") # 儲存 AI 計畫

    gr.Markdown(f"# 📅 課表 AI 提醒器 (V6.1 - 修正版)\n### 歡迎！目前顯示的是 **本週 ({WEEK_RANGE_STR})** 的課表。")

    with gr.Tabs():
        # --- Tab 1: 主頁 ---
        with gr.TabItem("🎓 本週總覽 & 每日課表"):
            gr.Markdown("## 🤖 AI 本週重點提醒")
            ai_summary_box = gr.Textbox(value=INITIAL_AI_SUMMARY_TEXT, label="AI 生成的本週提醒 (可手動編輯)", lines=12, interactive=True)
            save_btn = gr.Button("💾 儲存 AI 提醒 (儲存全文 + 細項到 GSheet)", variant="primary")
            save_status = gr.Markdown()
            gr.Markdown("---")
            gr.Markdown("## 🔍 查詢本週每日課程")
            day_selector = gr.Radio(choices=["一", "二", "三", "四", "五", "六", "日"], label="請選擇要查詢的星期", value=TODAY_WEEKDAY_STR)
            schedule_output = gr.DataFrame(
                label="當日課表",
                # (V6.1) 確保 Gradio 顯示的標頭是我們想要的
                headers=['課程名稱', '時間(起)', '時間(迄)', '地點', '攜帶品', '備註']
            )

        # --- Tab 2: 臨時 AI 編輯 (V5 保留) ---
        with gr.TabItem("✏️ 臨時 AI 編輯 (僅更新摘要)"):
            gr.Markdown(
                """
                ## ⚡ 動態為「本週」課程追加筆記 (僅供 AI 更新)
                在這裡輸入的資料**不會**儲存到 Google Sheet，
                但會**立即更新**「本週總覽」頁面中的 AI 摘要，方便您臨時加入提醒！
                """
            )
            course_selector = gr.Dropdown(choices=WEEK_COURSES_LIST, label="1. 選擇要追加筆記的課程")
            with gr.Row():
                new_item_box = gr.Textbox(label="2. 要追加的攜帶品 (例如: 剪刀)")
                new_note_box = gr.Textbox(label="3. 要追加的備註 (例如: 帶隨身碟)")
            update_ai_btn = gr.Button("🚀 更新 AI 摘要！", variant="primary")
            gr.Markdown("*(更新後請切回「本週總覽」分頁查看 AI 摘要的變化)*")

        # --- Tab 3: 永久新增 GSheet (V6 新增) ---
        with gr.TabItem("💾 永久新增 (GSheet 快速儲存)"):
            gr.Markdown(
                f"""
                ## ⚡ 永久新增「本週」({WEEK_RANGE_STR}) 紀錄
                在這裡輸入的資料**不會**更新 AI 摘要，
                但會**立即、永久**儲存到您 GSheet 的 `[攜帶物品]` 或 `[學習重點]` 分頁。
                """
            )
            gr.Markdown("### 🎒 新增 [攜帶物品]")
            direct_item_box = gr.Textbox(label="要新增的物品", placeholder="例如：麥克筆")
            direct_item_btn = gr.Button("💾 儲存物品")
            gr.Markdown("---")
            gr.Markdown("### 📚 新增 [備註/重點]")
            direct_note_box = gr.Textbox(label="要新增的備註", placeholder="例如：記得帶期中報告")
            direct_note_btn = gr.Button("💾 儲存備註")
            direct_save_status = gr.Markdown()

        # --- Tab 4: AI 學習計畫室 (V6 升級) ---
        with gr.TabItem("🚀 AI 學習計畫室"):
            gr.Markdown("## 💡 AI 課程外掛 (YouTube 推薦)")
            yt_course_selector = gr.Dropdown(choices=WEEK_COURSES_LIST, label="1. 選擇本週課程")
            yt_btn = gr.Button("幫我找 YouTube 推薦")
            yt_output = gr.Markdown()

            gr.Markdown("---")
            gr.Markdown("## 🎯 AI 個人計畫")
            goal_box = gr.Textbox(label="1. 輸入您的本週學習目標", placeholder="例如：我想把程式語言的 for 迴圈搞懂...")
            plan_btn = gr.Button("幫我生成個人計畫")
            plan_output = gr.Markdown()
            plan_save_btn = gr.Button("💾 儲存這個計畫到 GSheet")
            plan_save_status = gr.Markdown()

        # --- Tab 5: 原始資料 ---
        with gr.TabItem("📚 完整本週課表 (GSheet 原始資料)"):
            gr.DataFrame(value=DF_THIS_WEEK_ORIGINAL, label="本週所有課程")

    # --- 7. 綁定按鈕與函數 ---

    # 1. (主頁) 選擇星期
    day_selector.change(fn=filter_schedule_by_day, inputs=[day_selector, df_week_state], outputs=[schedule_output])

    # 2. (主頁) 儲存 AI 摘要
    save_btn.click(fn=save_all_summaries, inputs=[week_range_state, ai_summary_box, ai_summary_json_state], outputs=[save_status])

    # 3. (Tab 2) 臨時更新 AI
    update_ai_btn.click(fn=update_ai_summary_dynamically, inputs=[df_week_state, course_selector, new_item_box, new_note_box], outputs=[ai_summary_box, ai_summary_json_state])

    # 4. (Tab 3) 快速儲存物品
    direct_item_btn.click(fn=save_direct_entry, inputs=[week_range_state, gr.State("item"), direct_item_box], outputs=[direct_save_status])

    # 5. (Tab 3) 快速儲存備註
    direct_note_btn.click(fn=save_direct_entry, inputs=[week_range_state, gr.State("note"), direct_note_box], outputs=[direct_save_status])

    # 6. (Tab 4) YouTube 推薦 (V6.1 修正)
    yt_btn.click(
        fn=run_youtube_helper,
        inputs=[yt_course_selector],
        outputs=[yt_output]
    )

    # 7. (Tab 4) 個人計畫 (V6.1 修正)
    plan_btn.click(
        fn=run_plan_helper,
        inputs=[goal_box],
        outputs=[plan_output, plan_text_state] # 2 個輸出
    )

    # 8. (Tab 4) 儲存計畫
    plan_save_btn.click(
        fn=save_learning_plan,
        inputs=[goal_box, plan_text_state],
        outputs=[plan_save_status]
    )

    # 9. (頁面一載入)
    demo.load(
        fn=filter_schedule_by_day,
        inputs=[day_selector, df_week_state],
        outputs=[schedule_output]
    )

# --- 8. 啟動 Gradio 應用 ---
demo.launch(debug=True, inbrowser=True)

⏳ 正在安裝必要的套件 (gradio, gspread, google-generativeai...)...
✅ 所有套件安裝完成！
✅ 函式庫載入完成。
⏳ 正在進行 Google 認證...
✅ Colab 帳號認證成功 (Google Sheet)。
✅ Gemini AI 模型 (gemini-2.5-flash) 初始化成功。
⏳ (V4) 正在產生從 2025-11-03 到 2025-12-23 的「豐富版」課表...
✅ (V4) 成功產生 202 筆「豐富版」課程資料。
✅ 成功將 202 筆「V4 真實課表」資料覆蓋寫入 'HW6'！
⏳ 正在準備 Gradio 應用程式...
ℹ️ (偵錯) 已強制設定今天日期為: 2025-11-06
⏳ 正在啟動 Gradio 應用程式...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://aa8246508292f060a9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://aa8246508292f060a9.gradio.live
